<a href="https://colab.research.google.com/github/ndevar132/demo-repo/blob/master/tf_movie_review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

VOCAB_SIZE = 88584

MAXLEN = 250
BATCH_SIZE = 64

(train_data, train_labels), (test_data, test_labels) = imdb.load_data(num_words = VOCAB_SIZE)

Using TensorFlow backend.


In [2]:
len(train_data[0])

# preprocessing
train_data = sequence.pad_sequences(train_data, MAXLEN)
test_data = sequence.pad_sequences(test_data, MAXLEN)

# create model
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(VOCAB_SIZE, 32), # 32 => vector dimension
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation='sigmoid')
]
)

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 32)          2834688   
_________________________________________________________________
lstm (LSTM)                  (None, 32)                8320      
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 2,843,041
Trainable params: 2,843,041
Non-trainable params: 0
_________________________________________________________________


In [3]:
# training
model.compile(loss="binary_crossentropy", optimizer='rmsprop', metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=4, validation_split=0.2)

results = model.evaluate(test_data, test_labels)
print(results)

Epoch 1/4
625/625 [==============================] - 40s 63ms/step - loss: 0.4224 - acc: 0.8081 - val_loss: 0.2866 - val_acc: 0.8826
Epoch 2/4
625/625 [==============================] - 38s 61ms/step - loss: 0.2407 - acc: 0.9073 - val_loss: 0.2814 - val_acc: 0.8876
Epoch 3/4
625/625 [==============================] - 39s 62ms/step - loss: 0.1854 - acc: 0.9324 - val_loss: 0.2830 - val_acc: 0.8962
Epoch 4/4
782/782 [==============================] - 12s 16ms/step - loss: 0.3401 - acc: 0.8738
[0.34010741114616394, 0.8737999796867371]


In [15]:


# making prediction
word_index = imdb.get_word_index()

def encode_text(text):
  tokens = tf.keras.preprocessing.text.text_to_word_sequence(text)
  tokens = [word_index[word] if word in word_index else 0 for word in tokens]
  return sequence.pad_sequences([tokens], MAXLEN)[0]

text = "that movie was just amazing, so amazing"
encoded = encode_text(text)
print(encoded)

# decode function
reverse_word_index = {value: key for (key, value) in word_index.items()}

def decode_integers(integers):
  PAD = 0
  text = ""
  for num in integers:
    if num != PAD:
      text += reverse_word_index[num] + " "
    
  return text[:-1]

print(decode_integers(encoded))

# make prediction
def predict(text):
  encoded_text = encode_text(text)
  pred = np.zeros((1,250))
  pred[0] = encoded_text
  result = model.predict(pred)
  print(result[0])

positive_review = "That movie was so awesome! I really love it and would watch it again because it was amazingly great"
predict(positive_review)

negative_review = "that movie good. I hated it and wouldn't watch it again. Was one of the worst thing I've ever watched"
predict(negative_review)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0  12  17  13  4